In [ ]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
#from keras.datasets import mnist
#from keras.losses import mse, binary_crossentropy
#from keras.utils import plot_model
from keras import backend as K
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
#import argparse
import os
os.environ["THEANO_FLAGS"] = "device=cuda,exception_verbosity=high"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#IN_FILE = '/home/anil/Project/Ericsson/location/DPA_cellid_date_20170126.csv'
IN_FILE = 'DPA_cellid_date_20170126.csv'

#read data file
df = pd.read_csv(IN_FILE,header = 0,)

In [ ]:

#process the list of BTS for each user
df_ = df['cellid_set_count_1d_MOD'].str.split(',')
def get_bs(x):
    bs = []
    num = []
    try:
        for i in x:
            try:
                i_ = i.split(':')
                if (i_[0] != ''):
                    if (np.int(i_[1]) > 0):
                        bs.append(i_[0])
                        num.append(np.int(i_[1]))
            except:
                0
    except:
        0
    return bs
df_ = pd.DataFrame(df_)
#the column 'bss' contain the list of BTS for each user 
df_['bss'] = df_['cellid_set_count_1d_MOD'].apply(get_bs)
#filter out users with no BTSs
df_ = df_[df_.bss.apply(len) != 0]

df_user26 = df_.copy()

df_user26['bssCount'] = [len(c) for c in df_user26['bss']]

filter_users = list(df_user26[df_user26['bssCount']>=100].index)

df_user26_filtered = df_user26.loc[filter_users]

df_user26_filtered['bss_str'] = df_user26_filtered.bss.apply(lambda word_list : ' '.join(word_list))

#list of documents for each day
data26 = df_user26_filtered.loc[:,'bss_str'].tolist()

min_users = 100 #minimum users per bts

#vectorize the sing data to get the BSs that satisfy the minium user condition
tf_vectorizer26 = CountVectorizer(max_df=1.0, min_df=min_users, lowercase = False)
tf26 = tf_vectorizer26.fit_transform(data26) #Gives a sparse matrix

In [ ]:
U,B = tf26.shape

observed = tf26.T
# we can split observed into training and testing for prediction step
x_train = observed
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
input_shape = (U, )
batch_size = 512
latent_dim = 250
epochs = 50
encoding_dim = int(U/20)  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(encoding_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
#plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)
#%%
# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(encoding_dim, activation='relu')(latent_inputs)
outputs = Dense(U, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
#plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

def differences(pair_of_tensors):
    x, y = pair_of_tensors
    return K.abs(x - y)

def ham_loss(y_true,y_pred):
    # calculate pairwise hamming distance matrix
    # differences of y_pred 
    pairwise_diff_pred = K.abs(K.expand_dims(y_pred, 0) - K.expand_dims(y_pred, 1))
    pairwise_distance_pred = K.sum(pairwise_diff_pred, axis=-1)
    
    # calculate pairwise hamming distance matrix for inputs
    pairwise_diff_true = K.abs(K.expand_dims(y_true, 0) - K.expand_dims(y_true, 1))
    pairwise_distance_true = K.sum(pairwise_diff_true, axis=-1)
    
    #Difference between the distances of y_true and y_predictions
    hamm_sum= Lambda(differences)([pairwise_distance_true, pairwise_distance_pred])
    
    return K.sum(hamm_sum, axis=-1)
    
def vae_loss(y_true, y_pred):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z)]
    #recon = K.sum(mean_squared_error(y_true,y_pred))
    recon = K.sum(K.binary_crossentropy(y_true,y_pred),axis=1)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * K.sum(K.exp(z_log_var) + K.square(z_mean) - 1. - z_log_var, axis=1)
    
    hamming_loss = ham_loss(y_true,y_pred)

    return recon + kl + hamming_loss

In [ ]:
vae.compile(optimizer='adam', loss=vae_loss)
vae.fit(x_train, x_train, batch_size=batch_size, epochs=epochs)

In [ ]:
encoded_bts,_,_ = encoder.predict(x_train)
#print(encoded_bts.shape)
K_cluster = 100

##Fit GMM to the encoded graph
from sklearn import mixture
gmm = mixture.GaussianMixture(n_components= K_cluster, covariance_type='full')
y = gmm.fit(encoded_bts)

a_ = y.predict(encoded_bts)

gmm_clus = pd.DataFrame(a_, index = tf_vectorizer26.get_feature_names())
gmm_clus.to_csv('/raid60/anil.gaddam/vae_hamloss.csv')